In [ ]:
try:
    import cirq
    import cirq_google
    import numpy as np
    import math
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")
    import cirq
    import cirq_google
    import numpy as np
    import math

In [139]:
# # Taking the vector of integers as input
# arr = []
# n = int(input("Enter number of elements : "))
# for i in range(0, n):
#     arr.append(int(input()))
n = 6
arr = [1, 5, 2, 4, 10, 8]

In [140]:
# Convert a number into its binary representation
def binary(num, l):
    s = bin(num).split('0b')[1]
    while len(s) < l:
        s = '0' + s
    return s

In [141]:
# Check if all tupples of adjacent bits are different
def check(num, l):
    binary_val = binary(num, l)
    for i in range(1, len(binary_val)):
        if binary_val[i] == binary_val[i - 1]:
            return False
    return True

In [142]:
# Binary length of an integer
def bin_len(num):
    return int(math.ceil(math.log2(num)))

# Maximum binary length of a list of integers
def max_bin_len(nums):
    return int(max([bin_len(i) for i in nums]))

In [143]:
# List of indices of integers whose binary representation is such 
# that two adjacent bits are different
def get_list(nums):
    l = max_bin_len(nums)
    return [i for i in range(len(nums)) if check(nums[i], l)]

In [144]:
# Constructing the circuit
indices = get_list(arr)
number_of_qubits = bin_len(n)

In [145]:
# Create a vector representing the required state
q = cirq.LineQubit.range(number_of_qubits)
k = 1 / math.sqrt(len(indices))

params = []
for i in range(2 ** number_of_qubits):
    if i in indices:
        params.append(k)
    else:
        params.append(0)

In [146]:
# Creating the binary tree
tree = [0] * (2 ** (number_of_qubits + 1))

In [147]:
# Generating transition probabilities across the leaves
def generating_leaves(tree, leaves, n):
    # Generating leaves
    index = 2 ** n
    for i in range(len(leaves)):
        tree[index + i] = leaves[i]
    return tree

In [150]:
# Generating transition probabilities across the tree
def generation(tree, leaves, n):
    tree = generating_leaves(tree, leaves, n)
    for i in range(2 ** n - 1, 0, -1):
        l = tree[2 * i]
        r = tree[2 * i + 1]
        tree[i] = (l ** 2 + r ** 2) ** 0.5
        if tree[i] == 0:
            tree[2 * i] = 0
            tree[2 * i  + 1] = 0
        else:
            tree[2 * i] = l / tree[i]
            tree[2 * i + 1] = r / tree[i]
    return tree

In [151]:
tree = generation(tree, params, number_of_qubits)
print(tree)

[0, 0.9999999999999999, 0.7071067811865476, 0.7071067811865476, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0, 0, 1.0, 0.0, 0, 0]


In [152]:
def theta(l):
    return np.angle(complex(np.absolute(l), np.absolute((1 - l ** 2) ** 0.5)))

In [153]:
def cleaned_binary(ind):
    s = bin(ind).split('0b')[1]
    return [int(x) for x in list(s[1:])]

In [154]:
# Declaring the circuit
circ = cirq.Circuit()

In [155]:
print(cleaned_binary(6))
print(tree)
for ind, x in enumerate(tree):
    print(ind, x)

[1, 0]
[0, 0.9999999999999999, 0.7071067811865476, 0.7071067811865476, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0, 0, 1.0, 0.0, 0, 0]
0 0
1 0.9999999999999999
2 0.7071067811865476
3 0.7071067811865476
4 1.0
5 0.0
6 1.0
7 0.0
8 0.0
9 1.0
10 0
11 0
12 1.0
13 0.0
14 0
15 0


In [156]:
def construct(tree, number_of_qubits, qubits):
    # assuming that qubits are set to \ket{0}
    circ = cirq.Circuit()
    for x in qubits:
        cirq.reset(x)
        # print(cirq.Simulator().simulate(cirq.Circuit([cirq.measure(x, key='result')])))

    for index in range(1, 2 ** number_of_qubits):
        l = tree[2*index]
        angle = 2 *theta(l)/np.pi
        bits = cleaned_binary(index)

        # applying rotation
        height = int(np.floor(np.log2(index)))
        print(index, bits, angle, height, qubits[height], qubits[:height])
        if height == 0:
            circ.append(cirq.YPowGate(exponent=angle).on(qubits[0]))
        else:
            circ.append(cirq.YPowGate(exponent=angle).on(qubits[height]).controlled_by(*qubits[:height], control_values=bits))
    return circ

In [157]:
circ = construct(tree, number_of_qubits, q)
print(circ)

1 [] 0.5 0 0 []
2 [0] 0.0 1 1 [cirq.LineQubit(0)]
3 [1] 0.0 1 1 [cirq.LineQubit(0)]
4 [0, 0] 1.0 2 2 [cirq.LineQubit(0), cirq.LineQubit(1)]
5 [0, 1] 1.0 2 2 [cirq.LineQubit(0), cirq.LineQubit(1)]
6 [1, 0] 0.0 2 2 [cirq.LineQubit(0), cirq.LineQubit(1)]
7 [1, 1] 1.0 2 2 [cirq.LineQubit(0), cirq.LineQubit(1)]
0: ───Y^0.5───(0)───@─────(0)───(0)───@─────@───
              │     │     │     │     │     │
1: ───────────Y^0───Y^0───(0)───@─────(0)───@───
                          │     │     │     │
2: ───────────────────────Y─────Y─────Y^0───Y───


In [179]:
simulator = cirq.Simulator()
val = simulator.simulate(circ)
state = val.final_state_vector

In [180]:
# Required state
print(state)

[ 0. +0.j  -0.5+0.5j  0. +0.j   0. +0.j   0.5+0.5j  0. +0.j   0. +0.j
  0. +0.j ]
